函数调用功能使我们能够利用模型的自然语言理解能力，将人类语言有效地转化为结构化数据或在我们的代码中进行特定的函数调用。

In [1]:
function_descriptions = [
    {
        "name": "get_student_score",
        "description": "Get the student score by given his or her name",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The student's name",
                }
            },
            "required": ["name"],
        },
    }
]

In [2]:
user_query = "What's the performance of Lucy in the school this year?"

In [4]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
# os.environ["OPENAI_API_BASE"] = getpass.getpass()

api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_API_BASE")

print(api_key)
print(base_url)
client = OpenAI(
    api_key=api_key,
    base_url=base_url,
)

response = client.chat.completions.create(
    model="moonshot-v1-8k",
    messages=[{"role": "user", "content": user_query}],
    functions=function_descriptions,
    function_call="auto",
)

sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [5]:
ai_response_message = response["choices"][0]["message"]
print(ai_response_message)

TypeError: 'ChatCompletion' object is not subscriptable

In [8]:
response.choices[0].message

ChatCompletionMessage(content="As an AI, I don't have real-time access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. Therefore, I'm unable to provide information about Lucy's performance in school this year. However, if you have any general questions about education or studying techniques, feel free to ask!", role='assistant', function_call=None, tool_calls=None)

In [9]:
name = eval(ai_response_message['function_call']['arguments']).get("name")

NameError: name 'ai_response_message' is not defined

In [10]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_community.embeddings.cloudflare_workersai import CloudflareWorkersAIEmbeddings
from supabase.client import Client, create_client

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)
# embedding = CloudflareWorkersAIEmbeddings(
#     account_id=os.getenv('CF_ACCOUNT_ID'),
#     api_token=os.getenv('CF_API_TOKEN'),
#     model_name="@cf/baai/bge-small-en-v1.5",
# )
# 
# supabase_url = os.environ.get("SUPABASE_URL")
# supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
# 
# supabase: Client = create_client(supabase_url, supabase_key)
# 
# collection_name = 'yxk-robus-index'

/Users/pangmengting/venv/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! function is not default parameter.
                function was transferred to model_kwargs.
                Please confirm that function is what you intended.
  warnings.warn(


In [ ]:
qw_llm_openai.bind(parallel_tool_calls=False)

In [11]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage, SystemMessage

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "当你想知道现在的时间时非常有用。",
            "parameters": {},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "当你想查询指定城市的天气时非常有用。",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市或县区，比如北京市、杭州市、余杭区等。",
                    }
                },
            },
            "required": ["location"],
        },
    },
]

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the weather like in San Francisco?"),
]
chatLLM = ChatTongyi()
llm_kwargs = {"tools": tools, "result_format": "message"}
ai_message = chatLLM.bind(**llm_kwargs).invoke(messages)
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': '{"properties": {"location": "San Francisco"}, "type": "object"}'}, 'id': '', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '36b7affb-2f7a-9a83-84d4-365e3807214c', 'token_usage': {'input_tokens': 229, 'output_tokens': 28, 'total_tokens': 257}}, id='run-926ea4e3-cb94-46c9-bf4f-336a518444fc-0', tool_calls=[{'name': 'get_current_weather', 'args': {'properties': {'location': 'San Francisco'}, 'type': 'object'}, 'id': ''}])

In [12]:
ai_message2 = qw_llm_openai.bind(**llm_kwargs).invoke(messages)
ai_message2

TypeError: Completions.create() got an unexpected keyword argument 'function'

In [13]:
from langchain_core.pydantic_v1 import BaseModel, Field

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')
print(api_key)
print(base_url)

# openai/moonshot
ms_llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


llm_with_tools = ms_llm.bind_tools([GetWeather])

ai_msg = llm_with_tools.invoke(
    "what is the weather like in San Francisco",
)
ai_msg

sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


2024-07-08 10:31:17,077:INFO - HTTP Request: POST https://api.moonshot.cn/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'GetWeather:0', 'function': {'arguments': '{\n    "location": "San Francisco, CA"\n}', 'name': 'GetWeather'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 73, 'total_tokens': 88}, 'model_name': 'moonshot-v1-8k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-48f83d0a-d25d-4127-a51b-ebabf762ae0d-0', tool_calls=[{'name': 'GetWeather', 'args': {'location': 'San Francisco, CA'}, 'id': 'GetWeather:0'}], usage_metadata={'input_tokens': 73, 'output_tokens': 15, 'total_tokens': 88})

In [14]:
llm_with_tools2 = qw_llm_openai.bind_tools([GetWeather])

ai_msg2 = llm_with_tools2.invoke(
    "what is the weather like in San Francisco",
)
ai_msg2

TypeError: Completions.create() got an unexpected keyword argument 'function'

In [15]:
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI

account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')
print(account_id)
print(api_token)

# CloudflareWorkersAI
model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(
    account_id=account_id,
    api_token=api_token,
    model=model
)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn


In [16]:
llm_with_tools3 = cf_llm.bind_tools([GetWeather])

ai_msg3 = llm_with_tools3.invoke(
    "what is the weather like in San Francisco",
)
ai_msg3

AttributeError: 'CloudflareWorkersAI' object has no attribute 'bind_tools'